In [36]:
import requests
import pandas as pd
from datetime import datetime
import time

# we use this function to convert responses to dataframes
def df_from_response(res):
    # initialize temp dataframe for batch of data in response
    df = pd.DataFrame()

    # loop through each post pulled from res and append to df
    for post in res.json()['data']['children']:
        df = df.append({
            'title': post['data']['title'],
            'ups': post['data']['ups'],
            'comments': post['data']['num_comments'],
            'created_utc': datetime.fromtimestamp(post['data']['created_utc']).strftime('%Y-%m-%dT%H:%M:%SZ'),
            'id': post['data']['id'],
            'kind': post['kind']
        }, ignore_index=True)

    return df

# authenticate API
client_auth = requests.auth.HTTPBasicAuth('UiUdHhRW-Bt2Fe-2fcuAOQ', 'AFxJ8-6MOTXOJbvZjUM586QG_LctEg')
data = {
    'grant_type': 'password',
    'username': 'PapaTheButt',
    'password': 'Pedersen83'
}
headers = {'User-Agent': 'myBot/0.0.1'}

# send authentication request for OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=client_auth, data=data, headers=headers)
# extract token from response and format correctly
TOKEN = f"bearer {res.json()['access_token']}"
# update API headers with authorization (bearer token)
headers = {**headers, **{'Authorization': TOKEN}}

# initialize dataframe and parameters for pulling data in loop
data = pd.DataFrame()
params = {'limit': 100}

# loop through 10 times (returning 1K posts)
for i in range(4):
    # make request
    res = requests.get("https://oauth.reddit.com/r/news/new",
                       headers=headers,
                       params=params)

    # get dataframe from response
    new_df = df_from_response(res)
    # take the final row (oldest entry)
    row = new_df.iloc[len(new_df)-1]
    # create fullname
    fullname = row['kind'] + '_' + row['id']
    # add/update fullname in params
    params['after'] = fullname
    time.sleep(2)
    # append new_df to data
    data = data.append(new_df, ignore_index=True)

IndexError: single positional indexer is out-of-bounds

In [ ]:
data